In [17]:
!pip install pandas pyarrow ollama


  Using cached httpx-0.25.2-py3-none-any.whl.metadata (6.9 kB)
Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.26.0
    Uninstalling httpx-0.26.0:
      Successfully uninstalled httpx-0.26.0


## Import Ollama and load Mistral

In [24]:
import ollama
import pandas as pd


In [23]:
print("loading data ...")
df = pd.read_parquet("../docs/cnn/test-00000-of-00001.parquet")
row = df.loc[0]
fulltext = row['article']
humanmade = row['highlights']
print(row['article'])
print("---")
print(row['highlights'])

loading data ...
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesd

In [25]:
print("summarizing ...")
response = ollama.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': f"generate a short summary of the following sentence:{fulltext}",
  },
])
generated = response['message']['content']
print(generated)

summarizing ...
 The Palestinian Authority officially joined the International Criminal Court (ICC) as its 123rd member on Wednesday, granting the court jurisdiction over alleged crimes in Palestinian territories since 2014. The Palestinians' accession was marked with a ceremony at The Hague and followed their signing of the ICC's founding Rome Statute in January. Israel and the United States opposed this move due to their non-membership in the ICC. This development may lead to investigations into alleged war crimes against both Palestinians and Israelis, making Palestine subject to counter-charges as well. The ICC welcomed Palestine's decision to join the treaty and urged opposing governments to end pressure on Palestine. Prosecutor Fatou Bensouda will conduct a preliminary examination into war crimes committed during last summer's conflict between Israel and Hamas militants in Gaza, leaving more than 2,000 people dead. The ICC was established in 2002 to prosecute genocide, crimes aga

In [26]:
print (f"lenght of original full text: {len(fulltext)}")
print (f"lenght of generated summary : {len(generated)}")
print (f"lenght of human summary     : {len(humanmade)}")

lenght of original full text: 3612
lenght of generated summary : 1014
lenght of human summary     : 233


## Let's use some metric to evaluate the result

In [ ]:
!pip install rouge-score

## Let's create a wrong summary and a benchmark summary
The benchmark summary is created via chatgpt4

In [35]:
bad = "The boat travels the seas, there is no point in telling this story now though"
benchmark = """
The Palestinian Authority became the 123rd member of the International Criminal Court (ICC) on Wednesday, gaining the court's jurisdiction over alleged crimes in Palestinian territories. This accession, marked by a ceremony in The Hague, follows the Palestinians' signing of the ICC's Rome Statute in January and accepting its jurisdiction since June 13, 2014. The move, which Israel and the U.S. oppose, allows for potential war crimes investigations against Israelis, but also subjects Palestinians to possible counter-charges. Palestinian Foreign Minister Riad al-Malki hailed the step towards justice and peace, while the ICC emphasized the responsibilities that come with membership. Human Rights Watch supported Palestine's ICC membership, criticizing pressures against it. The ICC's preliminary examination, opened in January, may lead to formal investigations of both sides for alleged war crimes, including those committed during the 2014 Gaza conflict.
"""

In [31]:
from rouge_score import rouge_scorer


In [33]:

def calculateScore(metric,gen,ref,ben,bad):
    print(f"METRIC {metric}")

    rscorer = rouge_scorer.RougeScorer([metric], use_stemmer=True)
    sgener = rscorer.score ( ref , gen )[metric][2]
    sbench  = rscorer.score( ref , ben )[metric][2]
    sbad   = rscorer.score ( ref , bad )[metric][2]

    return {
        "metric": metric,
        "gen": sgener,
        "ben": sbench,
        "bad": sbad
    }

In [38]:
rouge3 = calculateScore("rouge3",generated,humanmade,benchmark,bad=bad)
print (rouge3)
rougeL = calculateScore("rougeL",generated,humanmade,benchmark,bad=bad)
print (rougeL)

METRIC rouge3
{'metric': 'rouge3', 'gen': 0.08602150537634409, 'ben': 0.09248554913294799, 'bad': 0.0}
METRIC rougeL
{'metric': 'rougeL', 'gen': 0.24210526315789474, 'ben': 0.22598870056497178, 'bad': 0.0816326530612245}


In [48]:
from evaluate import doEvaluate

doEvaluate(fulltext=fulltext,summary=generated,benchmark=benchmark)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <your Op*******************************var>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}